## Cálculo del R0 para una Corregimiento especifico de Panamá por partes.

In [1]:
# El modelo matemático está en:
# https://rpubs.com/Chris_666/631556
    
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")

link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
data = pd.read_excel(link,'TODO CORREGIMIENTO')

# eliminamos filas que no importan y asignamos la primera como cabecera
data.drop([0], inplace=True)
data.columns = data.iloc[0]
data.drop([1], inplace=True)
data.head(10)


1,ON,Ultimo,Indice,CD_Corr,Provincia,Distrito,Corregimiento,CD_Prov,Fecha,DIA,...,%Activos,%Nuevos Casos,%Nuevos Fallecidos,%Nuevos Recuperados,%Nuevos Activos,Casos/1MM hab,Fallecidos/1MM hab,Recuperados/1 MM hab,Activos/1MM hab,Población
2,ON,no,1,80821,Panamá,Panamá,24 de Diciembre,8,2020-06-05 00:00:00,1,...,0.393728,NaN,NaN,NaN,NaN,7193.16,200.506,4160.51,2832.15,79798
3,ON,no,2,30202,Colón,Chagres,Achiote,3,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,861
4,ON,no,3,70313,Los Santos,Los Santos,Agua Buena,7,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,1131
5,ON,no,4,120502,Comarca Ngäbe Buglé,Ñürün,Agua de Salud,12,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,2034
6,ON,no,5,50313,Darién,Santa Fe,Agua Fría,5,2020-06-05 00:00:00,1,...,1,NaN,NaN,NaN,NaN,333.444,0,0,333.444,2999
7,ON,no,6,20101,Coclé,Aguadulce,Aguadulce (Cabecera),2,2020-06-05 00:00:00,1,...,0.529412,NaN,NaN,NaN,NaN,1780.48,0,837.872,942.606,9548
8,ON,no,7,100102,Comarca Kuna Yala,Comarca Kuna Yala,Ailigandí,10,2020-06-05 00:00:00,1,...,0.0594059,NaN,NaN,NaN,NaN,7099.68,281.175,6396.74,421.763,14226
9,ON,no,8,40101,Chiriquí,Alanje,Alanje (Cabecera),4,2020-06-05 00:00:00,1,...,0.75,NaN,NaN,NaN,NaN,3234.94,0,808.734,2426.2,2473
10,ON,no,9,80822,Panamá,Panamá,Alcalde Díaz,8,2020-06-05 00:00:00,1,...,0.388235,NaN,NaN,NaN,NaN,5061.63,39.6991,3056.83,1965.1,50379
11,ON,no,10,10401,Bocas del Toro,Almirante,Almirante (Cabecera),1,2020-06-05 00:00:00,1,...,0.5,NaN,NaN,NaN,NaN,1244.17,0,622.084,622.084,3215


In [2]:

# extraemos solo las columnas que nos interesan
our_data = data[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]
our_data


1,CD_Corr,Corregimiento,Fecha,Nuevos Confirmados,Nuevos Fallecidos,Nuevos Recuperados,Población
2,80821,24 de Diciembre,2020-06-05 00:00:00,NaN,NaN,NaN,79798
3,30202,Achiote,2020-06-05 00:00:00,NaN,NaN,NaN,861
4,70313,Agua Buena,2020-06-05 00:00:00,NaN,NaN,NaN,1131
5,120502,Agua de Salud,2020-06-05 00:00:00,NaN,NaN,NaN,2034
6,50313,Agua Fría,2020-06-05 00:00:00,NaN,NaN,NaN,2999
...,...,...,...,...,...,...,...
70767,41401,Volcán (Cabecera),2020-09-17 00:00:00,0,0,0,4168
70768,50206,Yape,2020-09-17 00:00:00,0,0,0,216
70769,50207,Yaviza,2020-09-17 00:00:00,0,0,0,5128
70770,50317,Zapallal,2020-09-17 00:00:00,0,0,0,2072


In [4]:
# our_data_filter = our_data[our_data['CD_Corr'] == CD_Corr_unique[97]]
# hacemos una prueba para una comuna especifica
# our_data_filter = our_data[our_data['Corregimiento'] == 'Alanje (Cabecera)']
our_data_filter = our_data[our_data['Corregimiento'] == 'Salud']

# our_data_filter = our_data
our_data_filter


1,CD_Corr,Corregimiento,Fecha,Nuevos Confirmados,Nuevos Fallecidos,Nuevos Recuperados,Población
561,30207,Salud,2020-06-05 00:00:00,NaN,NaN,NaN,2415
1235,30207,Salud,2020-06-06 00:00:00,0,0,0,2415
1909,30207,Salud,2020-06-07 00:00:00,0,0,0,2415
2583,30207,Salud,2020-06-08 00:00:00,0,0,0,2415
3257,30207,Salud,2020-06-09 00:00:00,0,0,0,2415
...,...,...,...,...,...,...,...
67961,30207,Salud,2020-09-13 00:00:00,0,0,0,2415
68635,30207,Salud,2020-09-14 00:00:00,0,0,0,2415
69309,30207,Salud,2020-09-15 00:00:00,0,0,0,2415
69983,30207,Salud,2020-09-16 00:00:00,0,0,0,2415


In [5]:
# renombramos las columnas y las casteamos como float
our_data_filter.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)

our_data_filter['NuevosConfirmados'] = pd.to_numeric(our_data_filter['NuevosConfirmados'])
our_data_filter['NuevosFallecidos'] = pd.to_numeric(our_data_filter['NuevosFallecidos'])
our_data_filter['NuevosRecuperados'] = pd.to_numeric(our_data_filter['NuevosRecuperados'])
our_data_filter['Poblacion'] = pd.to_numeric(our_data_filter['Poblacion'])
our_data_filter

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion
561,30207,Salud,2020-06-05 00:00:00,NaN,NaN,NaN,2415
1235,30207,Salud,2020-06-06 00:00:00,0.0,0.0,0.0,2415
1909,30207,Salud,2020-06-07 00:00:00,0.0,0.0,0.0,2415
2583,30207,Salud,2020-06-08 00:00:00,0.0,0.0,0.0,2415
3257,30207,Salud,2020-06-09 00:00:00,0.0,0.0,0.0,2415
...,...,...,...,...,...,...,...
67961,30207,Salud,2020-09-13 00:00:00,0.0,0.0,0.0,2415
68635,30207,Salud,2020-09-14 00:00:00,0.0,0.0,0.0,2415
69309,30207,Salud,2020-09-15 00:00:00,0.0,0.0,0.0,2415
69983,30207,Salud,2020-09-16 00:00:00,0.0,0.0,0.0,2415


In [6]:
#eliminamos la primera fila que es un NA
our_data_filter = our_data_filter.iloc[1:]
our_data_filter

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion
1235,30207,Salud,2020-06-06 00:00:00,0.0,0.0,0.0,2415
1909,30207,Salud,2020-06-07 00:00:00,0.0,0.0,0.0,2415
2583,30207,Salud,2020-06-08 00:00:00,0.0,0.0,0.0,2415
3257,30207,Salud,2020-06-09 00:00:00,0.0,0.0,0.0,2415
3931,30207,Salud,2020-06-10 00:00:00,0.0,0.0,0.0,2415
...,...,...,...,...,...,...,...
67961,30207,Salud,2020-09-13 00:00:00,0.0,0.0,0.0,2415
68635,30207,Salud,2020-09-14 00:00:00,0.0,0.0,0.0,2415
69309,30207,Salud,2020-09-15 00:00:00,0.0,0.0,0.0,2415
69983,30207,Salud,2020-09-16 00:00:00,0.0,0.0,0.0,2415


In [7]:
# adjuntamos nuevas columnas con los valores necesarios para el calculo del R0:
our_data_filter['NuevosConfirmados'] = our_data_filter['NuevosConfirmados']+our_data_filter['NuevosFallecidos']
our_data_filter['NuevosConfirmadosacum'] = our_data_filter['NuevosConfirmados'].cumsum()

our_data_filter['NuevosRecuperadosacum'] = our_data_filter['NuevosRecuperados'].cumsum()

our_data_filter["Susceptibles"] = our_data_filter['Poblacion']-our_data_filter['NuevosConfirmadosacum']-our_data_filter['NuevosRecuperadosacum']
our_data_filter["pob_log_Susceptibles"] = our_data_filter['Poblacion']*np.log(our_data_filter["Susceptibles"])

our_data_filter.head(20)

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles
1235,30207,Salud,2020-06-06 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
1909,30207,Salud,2020-06-07 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
2583,30207,Salud,2020-06-08 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
3257,30207,Salud,2020-06-09 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
3931,30207,Salud,2020-06-10 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
4605,30207,Salud,2020-06-11 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
5279,30207,Salud,2020-06-12 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
5953,30207,Salud,2020-06-13 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
6627,30207,Salud,2020-06-14 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777
7301,30207,Salud,2020-06-15 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777


In [8]:
# extraemos un subconjunto que llegue solo hasta el dia de hoy:
import datetime
now = datetime.datetime.now()
our_data_filter_un = our_data_filter[our_data_filter['Fecha'] < now]
our_data_filter_un.tail(5) 

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles
59199,30207,Salud,2020-08-31 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016
59873,30207,Salud,2020-09-01 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016
60547,30207,Salud,2020-09-02 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016
61221,30207,Salud,2020-09-03 00:00:00,-9.0,0.0,-1.0,2415,0.0,0.0,2415.0,18811.532777
61895,30207,Salud,2020-09-04 00:00:00,0.0,0.0,0.0,2415,0.0,0.0,2415.0,18811.532777


In [9]:
# filtramos desde el primer dia en el que haya recuperados:

our_data_filter_un_final = our_data_filter_un[our_data_filter_un['NuevosRecuperadosacum'] > 0]
our_data_filter_un_final.tail(5)

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles
57851,30207,Salud,2020-08-29 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016
58525,30207,Salud,2020-08-30 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016
59199,30207,Salud,2020-08-31 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016
59873,30207,Salud,2020-09-01 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016
60547,30207,Salud,2020-09-02 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016


In [10]:
y = our_data_filter_un_final["pob_log_Susceptibles"]
y.head(5)

47067    18801.512016
47741    18801.512016
48415    18801.512016
49089    18801.512016
49763    18801.512016
Name: pob_log_Susceptibles, dtype: float64

In [54]:
# calculamos el R0
lista_R0 = []
e = []
from sklearn import linear_model
for i in range(len(y)):
    
    y = our_data_filter_un_final["pob_log_Susceptibles"]
    x = our_data_filter_un_final["NuevosRecuperadosacum"]

    #yy = y.head(i-1,i+1)
    #xx = x.head(i-1,i+1)
    
    ## recorrer las listas recortadas con el metodo drop
    #yyy = y.drop([1, 2], axis=0)
    
    # y.drop(range(47741,1000))
    
    # yy = y.drop([4, 5], axis=2)
   # invoices.drop(['client'], axis=1)
    ## seleccionar las dos primeras con el metodo head.

    
    #******************
    
    #
    # y.drop(y[y.X > 0].index, inplace=True)
    #yy = y.head(2)
    
    yy = y[i:len(y)]
    yyy = yy.head(2)
    
    xx = x[i:len(y)]
    xxx = xx.head(2)
    
    
    
    #****************** 
    # original
    # yy = y.head(i+1)
    # xx = x.head(i+1)
    #****************** 
    
    # Creamos el objeto de Regresión Lineal
    regresion_lineal = linear_model.LinearRegression()
    
    # regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
    regresion_lineal.fit(xxx.values.reshape(-1, 1),yyy.values.reshape(-1, 1))
    r0 = -regresion_lineal.coef_
    lista_R0.append(r0)
    
len(lista_R0)

our_data_filter_un_final["r0"] = lista_R0
# our_data_filter_un_final
our_data_filter_un_final


1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles,r0
47067,30207,Salud,2020-08-13 00:00:00,2.0,0.0,1.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
47741,30207,Salud,2020-08-14 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
48415,30207,Salud,2020-08-15 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
49089,30207,Salud,2020-08-16 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
49763,30207,Salud,2020-08-17 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
50437,30207,Salud,2020-08-18 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
51111,30207,Salud,2020-08-19 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
51785,30207,Salud,2020-08-20 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
52459,30207,Salud,2020-08-21 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]
53133,30207,Salud,2020-08-22 00:00:00,0.0,0.0,0.0,2415,9.0,1.0,2405.0,18801.512016,[[-0.0]]


In [215]:
# limpiamos los corchetes del R0:
our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
r0_puro = our_data_filter_un_final["r0"]
our_data_filter_un_final["r0"] = r0_puro
our_data_filter_un_final

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles,r0
28244,30207,Salud,2020-07-16 00:00:00,6.0,1.0,3.0,2415,25.0,3.0,2387.0,18783.369193,-0.000000
28918,30207,Salud,2020-07-17 00:00:00,0.0,0.0,0.0,2415,25.0,3.0,2387.0,18783.369193,-0.000000
29592,30207,Salud,2020-07-18 00:00:00,0.0,0.0,1.0,2415,25.0,4.0,2386.0,18782.357250,1.011942
30266,30207,Salud,2020-07-19 00:00:00,1.0,0.0,1.0,2415,26.0,5.0,2384.0,18780.332093,1.472495
30940,30207,Salud,2020-07-20 00:00:00,0.0,0.0,1.0,2415,26.0,6.0,2383.0,18779.318877,1.399685
31614,30207,Salud,2020-07-21 00:00:00,0.0,0.0,0.0,2415,26.0,6.0,2383.0,18779.318877,1.385634
32288,30207,Salud,2020-07-22 00:00:00,0.0,0.0,0.0,2415,26.0,6.0,2383.0,18779.318877,1.379663
32962,30207,Salud,2020-07-23 00:00:00,5.0,0.0,1.0,2415,31.0,7.0,2377.0,18773.230639,1.963507
33636,30207,Salud,2020-07-24 00:00:00,0.0,0.0,0.0,2415,31.0,7.0,2377.0,18773.230639,2.228891
34310,30207,Salud,2020-07-25 00:00:00,0.0,0.0,0.0,2415,31.0,7.0,2377.0,18773.230639,2.380539


In [ ]:
# Grabamos a un archivo local:
our_data_filter_un_final.to_csv (r'C:\Users\usuario\Desktop\R0_Panama.csv', mode='a', index = False, header=True)

## Cálculo del R0 para una Corregimiento especifico de Panamá integrado.

In [216]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")

link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
data = pd.read_excel(link,'TODO CORREGIMIENTO')

# eliminamos filas que no importan y asignamos la primera como cabecera
data.drop([0], inplace=True)
data.columns = data.iloc[0]
data.drop([1], inplace=True)
data.head(10)

# extraemos solo las columnas que nos interesan
our_data = data[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]

our_data_filter = our_data[our_data['Corregimiento'] == 'Salud']

# renombrar las columnas y castearlas como float
our_data_filter.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)

our_data_filter['NuevosConfirmados'] = pd.to_numeric(our_data_filter['NuevosConfirmados'])
our_data_filter['NuevosFallecidos'] = pd.to_numeric(our_data_filter['NuevosFallecidos'])
our_data_filter['NuevosRecuperados'] = pd.to_numeric(our_data_filter['NuevosRecuperados'])
our_data_filter['Poblacion'] = pd.to_numeric(our_data_filter['Poblacion'])

#eliminamos la primera fila que es un NA
our_data_filter = our_data_filter.iloc[1:]

our_data_filter['NuevosConfirmados'] = our_data_filter['NuevosConfirmados']+our_data_filter['NuevosFallecidos']
our_data_filter['NuevosConfirmadosacum'] = our_data_filter['NuevosConfirmados'].cumsum()

our_data_filter['NuevosRecuperadosacum'] = our_data_filter['NuevosRecuperados'].cumsum()

our_data_filter["Susceptibles"] = our_data_filter['Poblacion']-our_data_filter['NuevosConfirmadosacum']-our_data_filter['NuevosRecuperadosacum']
our_data_filter["pob_log_Susceptibles"] = our_data_filter['Poblacion']*np.log(our_data_filter["Susceptibles"])

# extraemos un subconjunto que llegue solo hasta el dia de hoy:
import datetime
now = datetime.datetime.now()
our_data_filter_un = our_data_filter[our_data_filter['Fecha'] < now]

# filtramos desde el primer dia en el que haya recuperados:

our_data_filter_un_final = our_data_filter_un[our_data_filter_un['NuevosRecuperadosacum'] > 0]

y = our_data_filter_un_final["pob_log_Susceptibles"]

lista_R0 = []
from sklearn import linear_model
for i in range(len(y)):
    y = our_data_filter_un_final["pob_log_Susceptibles"]
    x = our_data_filter_un_final["NuevosRecuperadosacum"]
    yy = y.head(i+1)
    xx = x.head(i+1)
    
    # Creamos el objeto de Regresión Lineal
    regresion_lineal = linear_model.LinearRegression()
    
    regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
    r0 = -regresion_lineal.coef_
    lista_R0.append(r0)
    
our_data_filter_un_final["r0"] = lista_R0

our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
r0_puro = our_data_filter_un_final["r0"]
our_data_filter_un_final["r0"] = r0_puro

our_data_filter_un_final.to_csv (r'C:\Users\usuario\Desktop\R0_Panama.csv', mode='a', index = False, header=True)

our_data_filter_un_final


1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles,r0
28244,30207,Salud,2020-07-16 00:00:00,6.0,1.0,3.0,2415,25.0,3.0,2387.0,18783.369193,-0.000000
28918,30207,Salud,2020-07-17 00:00:00,0.0,0.0,0.0,2415,25.0,3.0,2387.0,18783.369193,-0.000000
29592,30207,Salud,2020-07-18 00:00:00,0.0,0.0,1.0,2415,25.0,4.0,2386.0,18782.357250,1.011942
30266,30207,Salud,2020-07-19 00:00:00,1.0,0.0,1.0,2415,26.0,5.0,2384.0,18780.332093,1.472495
30940,30207,Salud,2020-07-20 00:00:00,0.0,0.0,1.0,2415,26.0,6.0,2383.0,18779.318877,1.399685
31614,30207,Salud,2020-07-21 00:00:00,0.0,0.0,0.0,2415,26.0,6.0,2383.0,18779.318877,1.385634
32288,30207,Salud,2020-07-22 00:00:00,0.0,0.0,0.0,2415,26.0,6.0,2383.0,18779.318877,1.379663
32962,30207,Salud,2020-07-23 00:00:00,5.0,0.0,1.0,2415,31.0,7.0,2377.0,18773.230639,1.963507
33636,30207,Salud,2020-07-24 00:00:00,0.0,0.0,0.0,2415,31.0,7.0,2377.0,18773.230639,2.228891
34310,30207,Salud,2020-07-25 00:00:00,0.0,0.0,0.0,2415,31.0,7.0,2377.0,18773.230639,2.380539


## Cálculo del R0 para todos los Corregimientos de Panamá.

In [38]:
# Código definitivo de cálculo de R0 para Panamá:

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")

link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
data = pd.read_excel(link,'TODO CORREGIMIENTO')

# eliminamos filas que no importan y asignamos la primera como cabecera
data.drop([0], inplace=True)
data.columns = data.iloc[0]
data.drop([1], inplace=True)
data.head(10)

# extraemos solo las columnas que nos interesan
our_data = data[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]

# nos preparamos para el ciclo sobre todos los Corregimientos:
CD_Corr_unique = np.unique(our_data['CD_Corr'])
cantidad_de_coor = len(CD_Corr_unique)

# hasta CD_Corr 400 ok
# cae en CD_Corr = 90608

# for i in range(cantidad_de_coor):

for i in range(cantidad_de_coor):

    our_data_filter = our_data[our_data['CD_Corr'] == CD_Corr_unique[i]]

    # our_data_filter = our_data[our_data['Corregimiento'] == 'Alanje (Cabecera)']

    # renombrar las columnas y castearlas como float
    our_data_filter.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)
    our_data_filter['NuevosConfirmados'] = pd.to_numeric(our_data_filter['NuevosConfirmados'])
    our_data_filter['NuevosFallecidos'] = pd.to_numeric(our_data_filter['NuevosFallecidos'])
    our_data_filter['NuevosRecuperados'] = pd.to_numeric(our_data_filter['NuevosRecuperados'])
    our_data_filter['Poblacion'] = pd.to_numeric(our_data_filter['Poblacion'])

    #eliminamos la primera fila que es un NA
    our_data_filter = our_data_filter.iloc[1:]

    our_data_filter['NuevosConfirmados'] = our_data_filter['NuevosConfirmados']+our_data_filter['NuevosFallecidos']
    our_data_filter['NuevosConfirmadosacum'] = our_data_filter['NuevosConfirmados'].cumsum()
    our_data_filter['NuevosRecuperadosacum'] = our_data_filter['NuevosRecuperados'].cumsum()
    our_data_filter["Susceptibles"] = our_data_filter['Poblacion']-our_data_filter['NuevosConfirmadosacum']-our_data_filter['NuevosRecuperadosacum']
    our_data_filter["pob_log_Susceptibles"] = our_data_filter['Poblacion']*np.log(our_data_filter["Susceptibles"])

    # extraemos un subconjunto que llegue solo hasta el dia de hoy:
    now = datetime.datetime.now()
    our_data_filter_un = our_data_filter[our_data_filter['Fecha'] < now]

    # filtramos desde el primer dia en el que haya recuperados:
    our_data_filter_un_final = our_data_filter_un[our_data_filter_un['NuevosRecuperadosacum'] > 0]
    
    y = our_data_filter_un_final["pob_log_Susceptibles"]

    # declaramos una lista para almacenar los R0:
    lista_R0 = []
    
    # Reemplazamos los NA por 0:
    our_data_filter_un_final.fillna("0", inplace = True) 
    
    for i in range(len(y)):
        try:
  
            # if (!np.isnan(y).bool() or  np.isinf(y).bool()):
            #    xx = np.nan_to_num(xx)
            y = our_data_filter_un_final["pob_log_Susceptibles"]
            x = our_data_filter_un_final["NuevosRecuperadosacum"]
            yy = y.head(i+1)
            xx = x.head(i+1)
            # print(xx)
            # print(yy)
            # Creamos el objeto de Regresión Lineal
            regresion_lineal = linear_model.LinearRegression()

            # if (np.isnan(xx).bool() or  np.isinf(xx).bool()):
            #    xx = np.nan_to_num(xx)

            regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
            r0 = -regresion_lineal.coef_
            lista_R0.append(r0)
            
        except:
            print("Error")
            lista_R0.append(0)
            
    our_data_filter_un_final["r0"] = lista_R0

    # our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
    # our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
    
    r0_puro = our_data_filter_un_final["r0"]
    our_data_filter_un_final["r0"] = r0_puro

    our_data_filter_un_final.to_csv (r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama.csv', header=False, mode='a', index = False)
    



Error
Error
Error
Error
Error
Error
Error


In [39]:
# Hay que agregar el código que sigue para eliminar los corchetes del r0
data_second_r0 = pd.read_csv(r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama.csv') 
data_second_r0

data_second_r0.columns = ['Código del Corregimiento','Corregimiento','Fecha','Casos Confirmados','Nuevos Fallecidos','Nuevos Recuperados','Poblacion','Nuevos Confirmados acum','Nuevos Recuperados acum','Susceptibles','pob_log_Susceptibles','r0']
#data_second_r0


x = data_second_r0['r0'].str.replace("]]", "")
y = x.str.replace("[", "")
z_dataframe = pd.DataFrame(y)
del data_second_r0['r0']
data_second_r0['r0'] = z_dataframe
#data_second_r0

data_second_r0.to_csv (r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama_final.csv', header=False, mode='a', index = False)


**********************************************
**********************************************
**********************************************

